In [ ]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 57.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import rasterio
import numpy as np

In [ ]:
#Monto mi drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# Insertar directorio donde se encuentran las imagenes opticas para agregarles los indices 
#directory_path = 
directory_files = os.listdir(directory_path)

In [ ]:
def indices(directory,path):
  """Agrega los indices ndvi y ndwi(ambas ecuaciones) como las ultimas 3 bandas al raster"""
  dataset = rasterio.open(directory+path)
  green = dataset.read(2)
  red = dataset.read(1)
  nir = dataset.read(4)
  swir = dataset.read(6)
  ndvi = (nir.astype(float) - red.astype(float))/(nir+red)
  ndwi_mcfeeters = (nir.astype(float) - green.astype(float))/(nir+green)
  ndwi_gao = (nir.astype(float) - swir.astype(float))/(nir+swir)
  profile = dataset.meta
  profile['count'] = dataset.count+3
  profile.update(driver='GTiff')
  profile.update(dtype=rasterio.float32)

  bands = dataset.read()
  bands = np.append(bands, np.array([ndvi, ndwi_mcfeeters,ndwi_gao]), axis = 0)
  with rasterio.open(directory+'/'+path+'', 'w', **profile) as dst:
    dst.write(bands,range(1,profile['count']+1))
  dataset.close()

In [ ]:
for each in directory_files:
  indices(directory_path,each)